# Homework 1

## FINM 36700 - 2024

### UChicago Financial Mathematics

* Mark Hendricks
* hendricks@uchicago.edu

## HBS Case

### *The Harvard Management Company and Inflation-Indexed Bonds*

### Notation
(Hidden LaTeX commands)

$$\newcommand{\mux}{\tilde{\boldsymbol{\mu}}}$$
$$\newcommand{\wtan}{\boldsymbol{\text{w}}^{\text{tan}}}$$
$$\newcommand{\wtarg}{\boldsymbol{\text{w}}^{\text{port}}}$$
$$\newcommand{\mutarg}{\tilde{\boldsymbol{\mu}}^{\text{port}}}$$
$$\newcommand{\wEW}{\boldsymbol{\text{w}}^{\text{EW}}}$$
$$\newcommand{\wRP}{\boldsymbol{\text{w}}^{\text{RP}}}$$
$$\newcommand{\wREG}{\boldsymbol{\text{w}}^{\text{REG}}}$$

***

# 1. HMC's Approach

**Section 1 is not graded**, and you do not need to submit your answers. But you are encouraged to think about them, and we will discuss them.

### 1. 
There are thousands of individual risky assets in which HMC can invest.  Explain why MV optimization across 1,000 securities is infeasible.

### 2.
Rather than optimize across all securities directly, HMC runs a two-stage optimization.
1. They build asset class portfolios with each one optimized over the securities of the specific asset class.  
2. HMC combines the asset-class portfolios into one total optimized portfolio.

In order for the two-stage optimization to be a good approximation of the full MV-optimization on all assets, what must be true of the partition of securities into asset classes?

### 3.
Should TIPS form a new asset class or be grouped into one of the other 11 classes?

### 4. 
Why does HMC focus on real returns when analyzing its portfolio allocation? Is this just a matter of scaling, or does using real returns versus nominal returns potentially change the MV solution?

### 5.
The case discusses the fact that Harvard places bounds on the portfolio allocation rather than implementing whatever numbers come out of the MV optimization problem.

How might we adjust the stated optimization problem in the lecture notes to reflect the extra constraints Harvard is using in their bounded solutions given in Exhibits 5 and 6?

### 6. 
Exhibits 5 shows zero allocation to domestic equities and domestic bonds across the entire computed range of targeted returns, (5.75% to 7.25%). Conceptually, why is the constraint binding in all these cases? What would the unconstrained portfolio want to do with those allocations and why?

### 7.
Exhibit 6 changes the constraints, (tightening them in most cases.) How much deterioration do we see in the mean-variance tradeoff that Harvard achieved?

### Answer:
1. We don't have enough samples to estimate n(n+1)/2 covariance and variance terms.
2. The securities within each asset class should hold enough similarity, and the size of securities in each asset class should be great enough to make sure the C.L.T converges to an average number. Additionally, it is also required to ensure that between-asset-classes covariance exsits, making the variance reduction meaningful.
3. New asses class
4. Yeah, it is because the TIPS is adjusted by inflation. Also, it can help measure the real purchasing power of each asset classes more reliable.
5. By adding the constraints that each asset class should have weight bigger than some value.
6. If no constrain, the MV will short sell domestic equities and domestic bonds.
7. Sharpe ratio drops 0.03 on average. 

***

# 2 Mean-Variance Optimization

<i>This section is graded for a good-faith effort by your group. Submit your write-up- along with your supporting code. </i>

### Data
You will need the file in the github repo, `data/multi_asset_etf_data.xlsx`.
- The time-series data gives monthly returns for the 11 asset classes and a short-term Treasury-bill fund return, ("SHV",) which we consider as the risk-free rate.
- The data is provided in total returns, (in which case you should ignore the SHV column,) as well as excess returns, (where SHV has been subtracted from the other columns.)
- These are nominal returns-they are not adjusted for inflation, and in our calculations we are not making any adjustment for inflation.
- The exhibit data that comes via Harvard with the case is unnecessary for our analysis.

### Model
We are going to analyze the problem in terms of **excess** returns.
- Thus, you will focus on the `Excess Returns` section of the lecture notes, especially the formulas on slide 50.
- Be sure to use the`excess returns` tab of the data.

### Format
In the questions below, **annualize the statistics** you report.
- Annualize the mean of monthly returns with a scaling of 12.
- Annualize the volatility of monthly returns with a scaling of $\sqrt{12}$
- The Sharpe Ratio is the mean return divided by the volatility of returns. Accordingly, we can annualize the Sharpe Ratio with a scaling of $\sqrt{12}$
- Note that we are not scaling the raw timeseries data, just the statistics computed from it (mean, vol, Sharpe). 

### Footnotes

#### Data File
* The case does not give time-series data, so this data has been compiled outside of the case, and it intends to represent the main asset classes under consideration via various ETFs. For details on the specific securities/indexes, check the “Info” tab of the data.

#### Risk-free rate
* In the lecture-note we considered a constant risk-free rate. It is okay that our risk-free rate changes over time, but the assumption is that investors know it’s value one-period ahead of time. Thus, at any given point in time, it is a risk-free rate for the next period. (This is often discussed as the "bank account" or "money market account" in other settings.

In [150]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")


df_description = pd.read_excel('../data/multi_asset_etf_data.xlsx', sheet_name = 'descriptions')
# display(df_description.head(3))

df_prices = pd.read_excel('../data/multi_asset_etf_data.xlsx', sheet_name = 'prices')
# display(df_prices.head(3))

df_total_returns = pd.read_excel('../data/multi_asset_etf_data.xlsx', sheet_name = 'total returns')
# display(df_total_returns.head(3))

df_excess_returns = pd.read_excel('../data/multi_asset_etf_data.xlsx', sheet_name = 'excess returns')
display(df_excess_returns.head(3))

,Date,BWX,DBC,EEM,EFA,HYG,IEF,IYR,PSP,QAI,SPY,TIP
0,2011-02-28,0.0065,0.0416,-0.0005,0.0354,0.0143,-0.0022,0.0451,0.0401,0.0025,0.0346,0.0071
1,2011-03-31,0.0083,0.0264,0.0627,-0.0241,0.0003,-0.0017,-0.0111,0.0165,0.0053,-0.0001,0.0117
2,2011-04-30,0.0483,0.0450,0.0268,0.0557,0.0154,0.0179,0.0461,0.0581,0.0185,0.0284,0.0232


## 1. Summary Statistics
* Calculate and display the mean and volatility of each asset’s excess return. (Recall we use volatility to refer to standard deviation.)
* Which assets have the best and worst Sharpe ratios? Recall that the Sharpe Ratio is simply the ratio of the mean-to-volatility of excess returns:
$$\text{sharpe ratio of investment }i = \frac{\mux_i}{\sigma_i}$$

## 2. Descriptive Analysis
* Calculate the correlation matrix of the returns. Which pair has the highest correlation? And the lowest?
* How well have TIPS done in our sample? Have they outperformed domestic bonds? Foreign bonds?

## 3. The MV frontier.
* Compute and display the weights of the tangency portfolios: $\wtan$.
* Does the ranking of weights align with the ranking of Sharpe ratios?
* Compute the mean, volatility, and Sharpe ratio for the tangency portfolio corresponding to
$\wtan$.

## 4. TIPS
Assess how much the tangency portfolio (and performance) change if...
* TIPS are dropped completely from the investment set.
* The expected excess return to TIPS is adjusted to be 0.0012 higher than what the historic sample shows.

Based on the analysis, do TIPS seem to expand the investment opportunity set, implying that Harvard should consider them as a separate asset?

In [151]:
df_excess_returns['Date'] = pd.to_datetime(df_excess_returns['Date'])
df_excess_returns.set_index('Date', inplace=True)
df_excess_returns.head(3)

,BWX,DBC,EEM,EFA,HYG,IEF,IYR,PSP,QAI,SPY,TIP
Date,,,,,,,,,,,
2011-02-28,0.0065,0.0416,-0.0005,0.0354,0.0143,-0.0022,0.0451,0.0401,0.0025,0.0346,0.0071
2011-03-31,0.0083,0.0264,0.0627,-0.0241,0.0003,-0.0017,-0.0111,0.0165,0.0053,-0.0001,0.0117
2011-04-30,0.0483,0.0450,0.0268,0.0557,0.0154,0.0179,0.0461,0.0581,0.0185,0.0284,0.0232


In [152]:
# 1
def summary_statistics(df):
    statistics = pd.DataFrame()
    statistics['Mean'] = np.mean(df, axis = 0) * 12
    statistics['Volatility'] = np.std(df, axis = 0, ddof=1) * np.sqrt(12)
    statistics['Sharpe'] = statistics['Mean'] / statistics['Volatility']

    return statistics

summary_table = summary_statistics(df_excess_returns)
display(summary_table)

print("Best Sharpe ratio is: ")
display(summary_table[summary_table['Sharpe'] == summary_table['Sharpe'].max()][['Sharpe']])
print("Worst Sharpe ratio is: ")
display(summary_table[summary_table['Sharpe'] == summary_table['Sharpe'].min()][['Sharpe']])

,Mean,Volatility,Sharpe
BWX,-0.0119,0.0817,-0.1456
DBC,-0.0091,0.1685,-0.0539
EEM,0.0270,0.1799,0.1498
EFA,0.0550,0.1522,0.3616
HYG,0.0374,0.0773,0.4833
IEF,0.0139,0.0632,0.2206
IYR,0.0779,0.1696,0.4594
PSP,0.0929,0.2152,0.4314
QAI,0.0150,0.0490,0.3052
SPY,0.1270,0.1431,0.8876


Best Sharpe ratio is: 


,Sharpe
SPY,0.8876


Worst Sharpe ratio is: 


,Sharpe
BWX,-0.1456


In [153]:
# 2
corr_table = df_excess_returns.corr()
display(corr_table)

corr_rank = corr_table.unstack().to_frame('Correlation')
corr_rank = df = corr_rank[~(corr_rank.index.get_level_values(0) == corr_rank.index.get_level_values(1))]

sorted_index = corr_rank.index.map(lambda x: tuple(sorted(x)))
corr_rank = corr_rank[~sorted_index.duplicated()]

print("Max Corr pair is: ")
display(corr_rank[corr_rank['Correlation'] == corr_rank['Correlation'].max()])
print("Min Corr pair is: ")
display(corr_rank[corr_rank['Correlation'] == corr_rank['Correlation'].min()])

,BWX,DBC,EEM,EFA,HYG,IEF,IYR,PSP,QAI,SPY,TIP
BWX,1.0000,0.2332,0.6387,0.6006,0.6209,0.5689,0.5626,0.5520,0.6624,0.4662,0.6800
DBC,0.2332,1.0000,0.5169,0.5214,0.4719,-0.3005,0.2900,0.4671,0.4945,0.4484,0.1144
EEM,0.6387,0.5169,1.0000,0.8208,0.6942,0.0205,0.5890,0.7618,0.7911,0.6999,0.3795
EFA,0.6006,0.5214,0.8208,1.0000,0.7907,0.0185,0.7059,0.9106,0.8640,0.8648,0.3867
HYG,0.6209,0.4719,0.6942,0.7907,1.0000,0.1801,0.7396,0.8142,0.8112,0.7948,0.5418
IEF,0.5689,-0.3005,0.0205,0.0185,0.1801,1.0000,0.2970,0.0175,0.1763,-0.0051,0.7450
IYR,0.5626,0.2900,0.5890,0.7059,0.7396,0.2970,1.0000,0.7538,0.7184,0.7606,0.5908
PSP,0.5520,0.4671,0.7618,0.9106,0.8142,0.0175,0.7538,1.0000,0.8707,0.8907,0.4159
QAI,0.6624,0.4945,0.7911,0.8640,0.8112,0.1763,0.7184,0.8707,1.0000,0.8625,0.5277
SPY,0.4662,0.4484,0.6999,0.8648,0.7948,-0.0051,0.7606,0.8907,0.8625,1.0000,0.3936


Max Corr pair is: 


,,Correlation
EFA,PSP,0.9106


Min Corr pair is: 


,,Correlation
DBC,IEF,-0.3005


In [154]:
# 3
def tangency_weight(returns):
    miu = np.mean(returns, axis = 0)

    inv_cov = np.linalg.inv(returns.cov() * 12)

    ones_col = np.ones(shape=(len(inv_cov), 1))
    first_term = 1/(ones_col.T @ inv_cov @ miu)[0]
    result_weight = first_term * inv_cov @ miu

    return result_weight

result_weight = tangency_weight(df_excess_returns)
result_table = pd.DataFrame(result_weight, columns = ['Weights'], index = summary_table.index)
result_table['Sharpe'] = summary_table['Sharpe']

result_table['Weights rank'] = result_table['Weights'].rank()
result_table['Sharpe rank'] = result_table['Sharpe'].rank()

display(result_table)
print("It is observed that the tangency porttfolio weight ranking is different from the sharpe ratio ranking")

def portfolio_statistics(df):
    statistics = pd.DataFrame()
    statistics['Mean'] = [np.mean(df) * 12]
    statistics['Volatility'] = [np.std(df, ddof = 1) * np.sqrt(12)]
    statistics['Sharpe'] = statistics['Mean'] / statistics['Volatility']

    return statistics

tangency_portfolio_return = portfolio_statistics(df_excess_returns @ result_weight)
display(tangency_portfolio_return)

,Weights,Sharpe,Weights rank,Sharpe rank
BWX,2.7303,-0.1456,10.0000,1.0000
DBC,-0.1112,-0.0539,5.0000,2.0000
EEM,-0.6157,0.1498,4.0000,3.0000
EFA,0.5307,0.3616,8.0000,7.0000
HYG,-0.8326,0.4833,3.0000,10.0000
IEF,-4.6825,0.2206,2.0000,4.0000
IYR,0.8771,0.4594,9.0000,9.0000
PSP,0.2770,0.4314,6.0000,8.0000
QAI,7.2206,0.3052,11.0000,5.0000
SPY,-4.9041,0.8876,1.0000,11.0000


It is observed that the tangency porttfolio weight ranking is different from the sharpe ratio ranking


,Mean,Volatility,Sharpe
0,-0.5273,0.3285,-1.6050


In [155]:
# 4
print("Weight without TIPS")
without_tips_weight = tangency_weight(df_excess_returns.copy().drop(['TIP'], axis = 1))
without_tips_table = pd.DataFrame(without_tips_weight, columns = ['Weights'], index = summary_table.index.drop('TIP'))
display(without_tips_table)

print("Performance without TIPS")
new_portfolio_return = portfolio_statistics(df_excess_returns.copy().drop(['TIP'], axis = 1) @ without_tips_weight)
display(new_portfolio_return)

Weight without TIPS


,Weights
BWX,3.0159
DBC,-0.0910
EEM,-0.6642
EFA,0.5406
HYG,-0.8716
IEF,-4.8324
IYR,0.9902
PSP,0.3249
QAI,7.9769
SPY,-5.3893


Performance without TIPS


,Mean,Volatility,Sharpe
0,-0.5808,0.3621,-1.6039


In [156]:
print("Weight with adjusted TIPS")
df_excess_returns_adjustedTIPS = df_excess_returns.copy()
df_excess_returns_adjustedTIPS['TIP'] += 0.0012
adjusted_tips_weight = tangency_weight(df_excess_returns_adjustedTIPS)
adjusted_tips_table = pd.DataFrame(adjusted_tips_weight, columns = ['Weights'], index = summary_table.index)
display(adjusted_tips_table)

print("Performance with adjusted TIPS")
new2_portfolio_return = portfolio_statistics(df_excess_returns_adjustedTIPS @ adjusted_tips_weight)
display(new2_portfolio_return)

Weight with adjusted TIPS


,Weights
BWX,29.2601
DBC,1.7716
EEM,-5.1230
EFA,1.4514
HYG,-4.4601
IEF,-18.6046
IYR,11.3761
PSP,4.7252
QAI,77.4621
SPY,-49.9636


Performance with adjusted TIPS


,Mean,Volatility,Sharpe
0,-6.1740,3.6558,-1.6888


In [157]:
alltogether_return = pd.concat([tangency_portfolio_return,new_portfolio_return,new2_portfolio_return])
alltogether_return.index = ['Tangent Weights', 'Dropped TIPS', 'Adjusted TIPS']
alltogether_return

,Mean,Volatility,Sharpe
Tangent Weights,-0.5273,0.3285,-1.6050
Dropped TIPS,-0.5808,0.3621,-1.6039
Adjusted TIPS,-6.1740,3.6558,-1.6888


### Conclusion

- Dropping TIPS doesn't affect much of the portfolio performance
- Adjust the TIPS return cause the Mean and Volatility change a lot

***

# 3. Allocations

<i>This section is graded for a good-faith effort by your group. Submit your write-up- along with your supporting code.

* Continue with the same data file as the previous section.

* Suppose the investor has a targeted mean excess return (per month) of $\mutarg$ = 0.01.

Build the following portfolios:

#### Equally-weighted (EW)
Rescale the entire weighting vector to have target mean $\mutarg$. Thus, the $i$ element of the weight vector is,
$$\wEW_i = \frac{1}{n}$$

#### “Risk-parity” (RP)
Risk-parity is a term used in a variety of ways, but here we have in mind setting the weight of the portfolio to be proportional to the inverse of its full-sample variance estimate. Thus, the $i$ element of the weight vector is,
$$\wRP_i = \frac{1}{\sigma_i^2}$$

#### Regularized (REG)
Much like the Mean-Variance portfolio, set the weights proportional to 
$$\wREG \sim \widehat{\Sigma}^{-1}\mux$$
but this time, use a regularized covariance matrix,
$$\widehat{\Sigma} = \frac{\Sigma + \Sigma_D}{2}$$
where $\Sigma_D$ denotes a *diagonal* matrix of the security variances, with zeros in the off-diagonals.

Thus, $\widehat{\Sigma}$ is obtained from the usual covariance matrix, $\Sigma$, but shrinking all the covariances to half their estimated values. 


### Comparing

In order to compare all these allocation methods, (those above, along with the tangency portfolio obtained in the previous section,) rescale each weight vector, such that it has targeted mean return of $\mutarg$.

* Calculate the performance of each of these portfolios over the sample.
* Report their mean, volatility, and Sharpe ratio. 
* How do these compare across the four allocation methods?

***

# 4. Out-of-Sample Performance

<i>This section is not graded, and you do not need to submit it. Still, we may discuss it in class, in which case, you would be expected to know it.

## 1. One-step Out-of-Sample (OOS) Performance
Let’s divide the sample to both compute a portfolio and then check its performance out of sample.
* Using only data through the end of `2022`, compute the weights built in Section 3.
* Rescale the weights, (using just the in-sample data,) to set each allocation to have the same mean return of $\mutarg$.
* Using those weights, calculate the portfolio’s Sharpe ratio within that sample.
* Again using those weights, (derived using data through `2022`,) calculate the portfolio’s OOS Sharpe ratio, which is based only on performance in `2023-2024`.

## 2. Rolling OOS Performance

Iterate the Out-of-Sample performance every year, not just the final year. Namely,
* Start at the end of `2015`, and calculate the weights through that time. Rescale them using the mean returns through that time.
* Apply the weights to the returns in the upcoming year, (`2016`.)
* Step forward a year in time, and recompute.
* Continue until again calculating the weights through `2023` and applying them to the returns in `2024`.

Report the mean, volatility, and Sharpe from this dynamic approach for the following portfolios:
* mean-variance (tangency)
* equally-weighted
* risk-parity
* regularized

***

# 5. Without a Riskless Asset

<i>This section is not graded, and you do not need to submit it. Still, we may discuss it in class, in which case, you would be expected to know it.

Re-do Section 2 above, but in the model without a risk-free rate.

That is, build the MV allocation using the two-part formula in the `Mean-Variance` section of the notes.
* This essentially substitutes the risk-free rate with the minimum-variance portfolio.
* Now, the allocation depends nonlinearly on the target mean return, $\mutarg$. (With a risk-free rate, we simply scale the weights up and down to achieve the mean return.)

You will find that, conceptually, the answers are very similar. 

***